# Grover's Search Algorithm - Quantum Search with Quadratic Speedup

Grover's algorithm is one of the most famous quantum algorithms, providing a quadratic speedup for unstructured search problems. This notebook explores the implementation and behavior of Grover's algorithm across different quantum simulators.

In [ ]:
# Import required libraries
import time

from ariadne import explain_routing, simulate
from ariadne.algorithms import AlgorithmParameters, GroverSearch

## Understanding Grover's Algorithm

Grover's algorithm searches an unstructured database of size N = 2^n to find a marked item:

**Classical complexity:** O(N) queries
**Quantum complexity:** O(√N) queries

Key components:
1. **Oracle**: Flips the phase of the marked state
2. **Diffusion operator**: Inverts about the mean amplitude
3. **Amplitude amplification**: Iteratively amplifies the marked state

## Creating Grover's Search Circuits

In [ ]:
# Create Grover circuits for different database sizes
grover_sizes = [2, 3, 4]  # 4, 8, 16 item databases
grover_circuits = {}

for n_qubits in grover_sizes:
    # Mark the last state (e.g., |11...1⟩) as the target
    marked_state = "1" * n_qubits
    params = AlgorithmParameters(n_qubits=n_qubits, custom_params={"marked_state": marked_state})

    grover = GroverSearch(params)
    circuit = grover.create_circuit()
    grover_circuits[n_qubits] = circuit

    print(f"Grover-{n_qubits} (searching in {2**n_qubits} items):")
    print(f"  Target state: |{marked_state}⟩")
    print(f"  Circuit: {circuit.num_qubits} qubits, {circuit.depth()} depth")
    print(f"  Gate counts: {circuit.count_ops()}")
    print()

## Visualizing Grover's Circuit Structure

In [ ]:
# Display the 3-qubit Grover circuit
print("3-qubit Grover Circuit (searching in 8 items):")
print(grover_circuits[3].draw(output="text"))

## Testing Grover's Algorithm Across Different Backends

In [ ]:
# Test 3-qubit Grover across different backends
test_circuit = grover_circuits[3]
backends = ["stim", "qiskit", "mps", "tensor_network"]
results = {}

print("Testing Grover-3 across backends:")
print("=" * 50)

for backend in backends:
    try:
        start_time = time.time()
        result = simulate(test_circuit, shots=1000, backend=backend)
        end_time = time.time()

        results[backend] = {
            "time": end_time - start_time,
            "counts": result.counts,
            "backend_used": result.backend_used.value,
        }

        print(f"{backend:15} | {end_time - start_time:8.4f}s | {result.backend_used.value}")

    except Exception as e:
        print(f"{backend:15} | FAILED - {str(e)[:30]}")
        results[backend] = {"error": str(e)}

## Analyzing Search Results

In [ ]:
# Analyze the success probability of finding the marked state
target_state = "111"  # Our marked state for 3-qubit case

print("\nGrover Search Success Analysis:")
print("=" * 40)

for backend, data in results.items():
    if "error" not in data:
        counts = data["counts"]
        total_shots = sum(counts.values())

        # Check success probability
        target_count = counts.get(target_state, 0)
        success_probability = target_count / total_shots

        print(f"\n{backend.upper()} Results:")
        print(f"  Target state |{target_state}⟩: {target_count:4d}/{total_shots} ({success_probability:.3f})")

        # Show top 3 most frequent outcomes
        sorted_counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)
        print("  Top 3 outcomes:")
        for _i, (outcome, count) in enumerate(sorted_counts[:3]):
            prob = count / total_shots
            marker = " ← TARGET" if outcome == target_state else ""
            print(f"    |{outcome}⟩: {count:4d} ({prob:.3f}){marker}")

## Scaling Analysis: How Performance Changes with Database Size

In [ ]:
# Test scaling behavior of Grover's algorithm
scaling_results = {}
test_sizes = [2, 3, 4]  # 4, 8, 16 item databases
test_backend = "qiskit"  # Use reliable backend for scaling test

print(f"Grover Scaling Analysis (backend: {test_backend}):")
print("=" * 60)
print(f"{'Qubits':<6} | {'Items':<6} | {'Depth':<6} | {'Time (s)':<9} | {'Success %':<9}")
print("-" * 45)

for n_qubits in test_sizes:
    circuit = grover_circuits[n_qubits]
    target_state = "1" * n_qubits

    try:
        start_time = time.time()
        result = simulate(circuit, shots=1000, backend=test_backend)
        end_time = time.time()

        # Calculate success probability
        counts = result.counts
        total_shots = sum(counts.values())
        target_count = counts.get(target_state, 0)
        success_rate = (target_count / total_shots) * 100

        print(
            f"{n_qubits:<6} | {2**n_qubits:<6} | {circuit.depth():<6} | {end_time - start_time:<9.4f} | {success_rate:<9.1f}"
        )

        scaling_results[n_qubits] = {
            "time": end_time - start_time,
            "depth": circuit.depth(),
            "success_rate": success_rate,
            "items": 2**n_qubits,
        }

    except Exception as e:
        print(f"{n_qubits:<6} | {2**n_qubits:<6} | FAILED - {str(e)[:20]}")

## Number of Iterations Analysis

In [ ]:
# Test how the number of Grover iterations affects success probability
def create_grover_with_iterations(n_qubits, iterations):
    """Create Grover circuit with specific number of iterations."""
    marked_state = "1" * n_qubits
    # This is a simplified version - in practice, you'd modify the algorithm
    params = AlgorithmParameters(
        n_qubits=n_qubits, custom_params={"marked_state": marked_state, "iterations": iterations}
    )

    grover = GroverSearch(params)
    return grover.create_circuit()


# Test different iteration counts for 3-qubit case
iteration_test = [1, 2, 3, 4]  # Different numbers of Grover iterations
n_qubits = 3
target_state = "111"

print("Effect of Grover Iterations (3-qubit case):")
print("=" * 50)
print(f"{'Iterations':<10} | {'Success %':<9} | {'Time (s)':<9}")
print("-" * 35)

for iterations in iteration_test:
    try:
        circuit = create_grover_with_iterations(n_qubits, iterations)

        start_time = time.time()
        result = simulate(circuit, shots=1000, backend="qiskit")
        end_time = time.time()

        # Calculate success probability
        counts = result.counts
        total_shots = sum(counts.values())
        target_count = counts.get(target_state, 0)
        success_rate = (target_count / total_shots) * 100

        print(f"{iterations:<10} | {success_rate:<9.1f} | {end_time - start_time:<9.4f}")

    except Exception as e:
        print(f"{iterations:<10} | FAILED - {str(e)[:20]}")

## Grover Circuit Analysis

In [ ]:
# Analyze circuit properties for different Grover sizes
print("\nGrover Circuit Properties:")
print("=" * 30)

for n_qubits in grover_sizes:
    marked_state = "1" * n_qubits
    params = AlgorithmParameters(n_qubits=n_qubits, custom_params={"marked_state": marked_state})

    grover = GroverSearch(params)
    analysis = grover.analyze_circuit_properties()

    print(f"\nGrover-{n_qubits} (searching in {2**n_qubits} items):")
    print(f"  Qubits: {analysis['n_qubits']}")
    print(f"  Depth: {analysis['depth']}")
    print(f"  Total gates: {analysis['size']}")
    print(f"  Two-qubit gates: {analysis['two_qubit_gates']}")
    print(f"  Entanglement heuristic: {analysis['entanglement_heuristic']:.2f}")
    print(f"  Gate types: {list(analysis['gate_counts'].keys())}")

## Educational Content: Grover's Mathematical Background

In [ ]:
# Get educational content about Grover's algorithm
marked_state = "111"
params = AlgorithmParameters(n_qubits=3, custom_params={"marked_state": marked_state})

grover = GroverSearch(params)
educational_content = grover.get_educational_content()

print("=== Grover's Mathematical Background ===")
print(educational_content["mathematical_background"])

print("\n=== Implementation Notes ===")
print(educational_content["implementation_notes"])

print("\n=== Applications ===")
print(educational_content["applications"])

## Routing Analysis for Grover's Algorithm

In [ ]:
# Analyze how Ariadne routes Grover circuits
print("Routing Analysis for Grover-3:")
print("=" * 30)

explanation = explain_routing(test_circuit)
print(explanation)

## Key Takeaways

1. **Quadratic Speedup**: Grover's algorithm finds items in O(√N) time vs O(N) classically
2. **Amplitude Amplification**: Success probability increases with optimal number of iterations
3. **Oracle Dependency**: The algorithm's efficiency depends on the oracle implementation
4. **Circuit Complexity**: Depth increases with the number of iterations and database size
5. **Backend Performance**: Different simulators handle the multi-controlled gates differently

Grover's algorithm demonstrates how quantum mechanics can provide provable speedups for fundamental computational problems, making it essential for quantum computing education and practical applications in search and optimization.